In [ ]:
import tools
import re
import numpy as np
import pandas as pd

from sklearn import linear_model, model_selection, preprocessing
from sklearn.metrics import roc_auc_score

In [ ]:
features_train = tools.features_train('data/features_train.csv')

In [ ]:
features_train.shape

(97230, 103)

In [ ]:
features_train.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0,1
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0,0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0,0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0,0


In [ ]:
features_train.tail()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,radiant_win
match_id,,,,,,,,,,,,,,,,,,,,,
114402,1450265551,1,47,4,1706,1198,17,0,1,8,...,-29.0,180.0,-76.0,180.0,3,4,3,0,-24.0,0
114403,1450277704,0,43,4,1793,1416,17,0,1,5,...,-5.0,NaN,-82.0,NaN,4,3,2,0,-17.0,1
114404,1450291848,1,98,4,1399,540,1,0,0,5,...,-32.0,249.0,-70.0,NaN,1,1,3,1,-15.0,0
114405,1450292986,1,100,3,1135,766,6,0,2,6,...,-21.0,254.0,-85.0,183.0,5,3,3,1,-42.0,0
114406,1450313370,7,50,3,1053,799,7,0,0,7,...,-36.0,131.0,-59.0,NaN,5,3,2,0,-39.0,1


In [ ]:
X, y = features_train.iloc[:, :-1].fillna(0), features_train.loc[:, 'radiant_win']

In [ ]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)

C:\Users\grum2\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\grum2\Anaconda3\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
X.shape, y.shape

((97230, 102), (97230,))

In [ ]:
cv_scores = []
for C in np.logspace(-5, 5, 11):
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2)
    clf = linear_model.LogisticRegression(C=C, random_state=2, solver='lbfgs')
    print('C =', str(C))
    
    start = pd.datetime.now()
    model_score = model_selection.cross_val_score(clf, X, y, cv=kf, scoring='roc_auc')
    print('Time elapsed:', pd.datetime.now() - start, '\n', model_score)
    
    cv_scores.append(model_score.mean())

C = 1e-05
Time elapsed: 0:00:04.226290 
 [0.69149974 0.69924063 0.70021953 0.69048894 0.69380073]
C = 0.0001
Time elapsed: 0:00:04.975283 
 [0.70846634 0.71294013 0.71603157 0.70693667 0.71112887]
C = 0.001
Time elapsed: 0:00:07.141417 
 [0.71421344 0.71570131 0.72071596 0.71284226 0.71658924]
C = 0.01
Time elapsed: 0:00:08.731104 
 [0.71450104 0.71532972 0.72103104 0.71324001 0.71685005]
C = 0.1
Time elapsed: 0:00:11.952299 
 [0.71448958 0.71524685 0.72105458 0.71322745 0.71685042]
C = 1.0
Time elapsed: 0:00:10.739859 
 [0.71448698 0.71523875 0.72105463 0.71322264 0.71684653]
C = 10.0
Time elapsed: 0:00:11.440001 
 [0.71448663 0.71523721 0.72105381 0.71322245 0.71684627]
C = 100.0
Time elapsed: 0:00:11.298716 
 [0.71448682 0.71523713 0.72105375 0.71322247 0.71684633]
C = 1000.0
Time elapsed: 0:00:12.170592 
 [0.71448689 0.71523703 0.72105375 0.71322254 0.71684637]
C = 10000.0
Time elapsed: 0:00:11.481090 
 [0.71448692 0.71523701 0.72105375 0.71322253 0.71684637]
C = 100000.0
Time elap